In [1]:
import gymnasium
from gymnasium import spaces
import numpy as np
import pandas as pd
import heapq
import matplotlib.pyplot as plt
from collections import defaultdict
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
import os
import random
import time

class RadarTaskSchedulerEnv(gymnasium.Env):
    def __init__(self, simulation_duration_minutes=60, render_mode=None):
        super(RadarTaskSchedulerEnv, self).__init__()

        self.simulation_duration_minutes = simulation_duration_minutes # Total simulation time for an episode
        self.current_simulation_time = 0.0 # Tracks the current time in the simulation

        # --- Radar Capabilities (Same as dataset generation) ---
        self.RADAR_CAPABILITIES = {
            'Radar1': ['Search', 'Detection', 'Classification'],
            'Radar2': ['Classification', 'Tracking', 'Detection'],
            'Radar3': ['Tracking', 'Search', 'Detection']
        }
        self.ALL_TASK_TYPES = ['Search', 'Detection', 'Tracking', 'Classification']

        # --- Stage Progression Logic (Same as dataset generation) ---
        self.MAX_STAGES_PER_TARGET_TYPE = {
            'Search': 1,        # 1 Search task is enough to potentially move to Detection
            'Detection': 4,     # Need 4 Detection tasks to move to Tracking
            'Tracking': 5,      # Need 5 Tracking tasks to move to Classification
            'Classification': 6 # Need 6 Classification tasks for 'Target Locked'
        }
        self.STAGE_ORDER = ['Search', 'Detection', 'Tracking', 'Classification', 'Locked']

        # --- Task Parameter Generation Ranges (Aligned with dataset generation) ---
        self.PRIORITY_MAP = {
            'Search': 1,
            'Detection': 2,
            'Tracking': 3,
            'Classification': 4
        }
        self.DEFAULT_PRIORITY_RANGE = (1, 5)

        self.AVG_TARGET_ARRIVAL_INTERVAL = 5.0 # Minutes between new target appearances

        self.DURATION_MAP = {
            'Search': (1, 3),
            'Detection': (2, 4),
            'Tracking': (3, 6),
            'Classification': (4, 8)
        }

        self.SLACK_FOR_DEADLINE_MAP = {
            'Search': (5, 15),
            'Detection': (4, 12),
            'Tracking': (3, 10),
            'Classification': (2, 8)
        }

        self.INITIAL_POWER_MAP = {
            'Search': (10, 20),
            'Detection': (20, 35),
            'Tracking': (30, 45),
            'Classification': (40, 60)
        }

        self.ADDITIONAL_MAX_POWER_RANGE = (0, 15)

        # --- Environment State ---
        self.event_queue = [] # (event_time, event_type, data) - for future target arrivals and task completions
        self.task_queue = [] # (priority, entry_count, task_dict) - tasks ready for scheduling (agent's choice)
        self.task_entry_count = 0 # Unique ID for task queue entries

        self.next_target_id = 1 # Counter for new target IDs

        # Key: Target_ID, Value: {'current_stage': 'Search', 'Search_count': 0, ..., 'last_processed_time': 0.0, 'is_locked': False}
        self.target_states = {}

        self.scheduled_tasks_log = [] # Log of all tasks processed in the current episode
        self.dropped_tasks_in_episode = 0 # Track dropped tasks

        self.current_task_for_observation = None # The task that the observation currently represents

        # Observation Space: [Duration, Deadline_Relative_to_Current_Time, Init_Power]
        # Max values are rough estimates, adapt if your ranges are different.
        # Deadline_Relative_to_Current_Time can be negative if already overdue.
        self.observation_space = spaces.Box(low=np.array([0, -self.simulation_duration_minutes, 0]),
                                            high=np.array([10, self.simulation_duration_minutes, 100]),
                                            shape=(3,), dtype=np.float32)

        # Action Space: 3 (Delay: 0, 1, 2) * 3 (Compress: 0, 1, 2) * 3 (Radar Choice: 0, 1, 2) = 27 actions
        self.action_space = spaces.Discrete(3 * 3 * 3)

        self.render_mode = render_mode
        self._last_info = {}
        self._last_reward = 0

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)

        self.current_simulation_time = 0.0
        self.event_queue = []
        self.task_queue = []
        self.task_entry_count = 0
        self.next_target_id = 1
        self.target_states = {}
        self.scheduled_tasks_log = []
        self.dropped_tasks_in_episode = 0
        self.current_task_for_observation = None
        self._last_info = {}
        self._last_reward = 0

        # Schedule the first target arrival event
        first_arrival_time = self.current_simulation_time + random.uniform(0.1, self.AVG_TARGET_ARRIVAL_INTERVAL * 1.5)
        heapq.heappush(self.event_queue, (first_arrival_time, 'TARGET_ARRIVAL', {'target_id': self.next_target_id}))
        self.next_target_id += 1

        # Advance simulation time until a task is ready or simulation ends
        self._advance_simulation_to_next_event()

        obs = self._get_observation()
        info = self._get_info()
        return obs, info

    def _advance_simulation_to_next_event(self):
        """
        Processes events from the event queue until a task is available for the agent
        or the simulation duration is exceeded.
        """
        while self.event_queue and not self.task_queue and self.current_simulation_time < self.simulation_duration_minutes:
            event_time, event_type, event_data = heapq.heappop(self.event_queue)

            if event_time > self.simulation_duration_minutes:
                # Event is beyond simulation duration, don't process
                continue

            self.current_simulation_time = max(self.current_simulation_time, event_time)

            if event_type == 'TARGET_ARRIVAL':
                self._handle_target_arrival(event_data['target_id'])
                # Schedule next target arrival
                if self.current_simulation_time < self.simulation_duration_minutes:
                    next_arrival_time = self.current_simulation_time + random.uniform(
                        0.5 * self.AVG_TARGET_ARRIVAL_INTERVAL,
                        1.5 * self.AVG_TARGET_ARRIVAL_INTERVAL
                    )
                    if next_arrival_time < self.simulation_duration_minutes:
                        heapq.heappush(self.event_queue, (next_arrival_time, 'TARGET_ARRIVAL', {'target_id': self.next_target_id}))
                        self.next_target_id += 1

            elif event_type == 'TASK_COMPLETION':
                self._handle_task_completion_event(event_data)
        
        # Ensure current_simulation_time doesn't exceed duration if no more events
        if not self.event_queue and not self.task_queue:
            self.current_simulation_time = self.simulation_duration_minutes


    def _handle_target_arrival(self, target_id):
        """Initializes a new target and generates its first 'Search' task."""
        self.target_states[target_id] = {
            'current_stage': 'Search',
            'Search_count': 0,
            'Detection_count': 0,
            'Tracking_count': 0,
            'Classification_count': 0,
            'last_processed_time': self.current_simulation_time, # When this target became active
            'is_locked': False
        }
        # Generate the first search task for this new target
        self._generate_and_queue_task_for_target(target_id)

    def _handle_task_completion_event(self, completed_task_data):
        """
        Handles the internal completion of a task, which might trigger a new task
        for the same target.
        """
        target_id = completed_task_data['target_id']
        task_type = completed_task_data['task_type']
        
        target_state = self.target_states.get(target_id)
        if not target_state or target_state['is_locked']:
            return # Target already locked or doesn't exist (shouldn't happen for completion event)

        # Assuming completion means success for triggering next stage
        # (Reward logic already handles dropped/successful)
        # Check if this completion fulfills the requirements for the next stage
        # The stage_count for the task in completed_task_data is the one *after* processing
        
        if task_type in self.MAX_STAGES_PER_TARGET_TYPE: # Only stages that lead to progression
            required_count_for_next_stage = self.MAX_STAGES_PER_TARGET_TYPE[task_type]

            # If the current task type count in the state has reached the requirement AND
            # the task just completed was the last one for this stage before transitioning
            if target_state[f'{task_type}_count'] >= required_count_for_next_stage and \
               target_state['current_stage'] == task_type: # Ensure we are at this stage
                
                current_stage_index = self.STAGE_ORDER.index(task_type)
                if current_stage_index + 1 < len(self.STAGE_ORDER):
                    next_stage_type = self.STAGE_ORDER[current_stage_index + 1]
                    
                    if next_stage_type == 'Locked':
                        target_state['is_locked'] = True
                        target_state['current_stage'] = 'Locked' # Update state
                        return # Target is now locked, no new tasks
                    
                    # Transition the target to the next stage
                    target_state['current_stage'] = next_stage_type
                    # Do NOT reset the count for the new stage; it starts counting from 0 (implicitly)
                    # and the _generate_and_queue_task_for_target will increment it

        # Always try to generate a task for the target's current stage
        self._generate_and_queue_task_for_target(target_id)


    def _generate_and_queue_task_for_target(self, target_id):
        """
        Generates a task for the given target_id based on its current state and queues it.
        This is called for initial 'Search' tasks and subsequent tasks upon successful completion.
        """
        target_state = self.target_states.get(target_id)
        if not target_state or target_state['is_locked']:
            return # Target already locked

        task_type = target_state['current_stage']
        
        # Increment the count for the stage type we are about to generate
        target_state[f'{task_type}_count'] += 1
        current_task_stage_count = target_state[f'{task_type}_count']

        # Ensure we don't generate more tasks for a stage than specified in MAX_STAGES_PER_TARGET_TYPE,
        # unless it's a progression to the next stage which is handled by _handle_task_completion_event
        if task_type != 'Search' and current_task_stage_count > self.MAX_STAGES_PER_TARGET_TYPE[task_type]:
            # This handles cases where a target completed enough of a previous stage,
            # but we've already generated all the tasks for this new stage type.
            # This logic needs to be careful; the main progression should happen in _handle_task_completion_event
            # based on *successful* processing.
            # If we're here, it means we've generated enough for THIS stage already,
            # and _handle_task_completion_event should have moved it forward if applicable.
            return # No more tasks for this stage type

        # Generate task parameters
        request_time = self.current_simulation_time # Task is requested now
        
        duration_range = self.DURATION_MAP.get(task_type, (2, 8))
        duration = random.randint(duration_range[0], duration_range[1])

        slack_range = self.SLACK_FOR_DEADLINE_MAP.get(task_type, (3, 10))
        slack_for_deadline = random.randint(slack_range[0], slack_range[1])
        deadline = round(request_time + duration + slack_for_deadline, 2)

        initial_power_range = self.INITIAL_POWER_MAP.get(task_type, (10, 50))
        initial_power = random.randint(initial_power_range[0], initial_power_range[1])
        max_power = initial_power + random.randint(self.ADDITIONAL_MAX_POWER_RANGE[0], self.ADDITIONAL_MAX_POWER_RANGE[1])

        max_delay = round(deadline - request_time - duration, 2)
        if max_delay < 0: max_delay = 0.0

        eligible_radars = [r for r, caps in self.RADAR_CAPABILITIES.items() if task_type in caps]
        radar_type = random.choice(eligible_radars) if eligible_radars else "AnyRadar"

        priority = self.PRIORITY_MAP.get(task_type, random.randint(self.DEFAULT_PRIORITY_RANGE[0], self.DEFAULT_PRIORITY_RANGE[1]))

        new_task = {
            'Target_ID': target_id,
            'Task_ID': f"{target_id}_{task_type[0]}{current_task_stage_count}",
            'Task_Type': task_type,
            'Radar_Type': radar_type,
            'Priority': priority,
            'Request_Time': request_time,
            'Deadline': deadline,
            'Duration': duration,
            'Init_Power': initial_power,
            'Max_Power': max_power,
            'Max_Delay': max_delay,
            'Stage_Count': current_task_stage_count
        }
        
        # Push to the task queue for the agent to observe and act upon
        negative_priority = -new_task['Priority']
        heapq.heappush(self.task_queue, (negative_priority, self.task_entry_count, new_task))
        self.task_entry_count += 1


    def step(self, action_index):
        reward = 0.0
        truncated = False
        done = False

        if not self.task_queue:
            # No tasks currently available, advance simulation time
            self._advance_simulation_to_next_event()
            if not self.task_queue: # Still no tasks after advancing time
                # Simulation ends if no more tasks and simulation duration is met or exceeded
                if self.current_simulation_time >= self.simulation_duration_minutes:
                    done = True
                    self._last_info['message'] = "Simulation duration reached with no tasks."
                else:
                    # If still within duration but no tasks, wait for next event (should be handled by advance_simulation)
                    # This case implies agent might need to take 'no-op' action or a different observation state.
                    # For now, let's just penalize slightly and continue, or handle as a done state if needed.
                    # For simplicity, returning a zero obs and small negative reward
                    reward -= 0.1 # Penalty for idle time
                    self._last_info['message'] = "No tasks available, advancing time."
                    return self._get_observation(), reward, done, truncated, self._get_info()

        # If a task is now available (either it was already there or _advance_simulation_to_next_event found one)
        _priority, _entry_count, task_to_process = heapq.heappop(self.task_queue)
        
        # Action Decoding
        delay_action = action_index // 9 # 0, 1, 2 (corresponding to 0, 1, 2 minutes delay)
        compress_action = (action_index % 9) // 3 # 0, 1, 2 (corresponding to 0%, 10%, 20% compression)
        radar_choice_action = action_index % 3 # 0, 1, 2 (for 3 radars)

        # Calculate actual TNS and Pn based on actions
        # TNS (Task Nominal Start Time) = Request_Time + Delay_Action_Minutes
        calculated_tns = task_to_process['Request_Time'] + delay_action
        
        # Pn (Power Nominal) = Initial_Power + (Max_Power - Initial_Power) * (Compress_Action / 2.0)
        # Assuming compress_action 0 = no additional power, 1 = 50% of additional_max_power, 2 = 100%
        # This mapping is arbitrary, adjust as per real requirements.
        additional_power_from_compression = (task_to_process['Max_Power'] - task_to_process['Init_Power']) * (compress_action / 2.0)
        calculated_Pn = task_to_process['Init_Power'] + additional_power_from_compression

        if compress_action == 0:
            power_increase_from_compression = 0.0
        elif compress_action == 1:
            power_increase_from_compression = 0.15 * task_to_process['Init_Power'] # Example: 15% of initial power
        elif compress_action == 2:
            power_increase_from_compression = 0.30 * task_to_process['Init_Power'] # Example: 30% of initial power
        else: # Should not happen with current action space
            power_increase_from_compression = 0.0

        calculated_Pn = task_to_process['Init_Power'] + power_increase_from_compression

        # Check for task drop conditions
        task_dropped = False
        if (calculated_tns + task_to_process['Duration']) > task_to_process['Deadline']:
            task_dropped = True
        if calculated_Pn > task_to_process['Max_Power']:
            task_dropped = True
        
        # Simulate radar selection (though not used for drop logic currently, could be added)
        # For simplicity, we just assume the chosen radar can perform the task if eligible_radars had choices.
        # In a more complex env, specific radar availability or capacity could be modeled.
        
        # --- Reward Calculation ---
        reward = 0.0

        if not task_dropped:
            reward += 1.0 # Base reward for successful completion

            # Time-based reward
            time_remaining = task_to_process['Deadline'] - calculated_tns
            if time_remaining >= 0:
                reward += time_remaining * 0.05 # Bonus for time remaining
            else: # Should be caught by task_dropped, but as a safeguard
                reward -= 0.5

            # Power-based reward
            power_margin = task_to_process['Max_Power'] - calculated_Pn
            if power_margin >= 0:
                reward += power_margin * 0.01 # Bonus for power efficiency
            else:
                reward -= 0.5

            # Incentive for minimal delay/compression if successful
            if delay_action == 0:
                reward += 0.1
            if compress_action == 0:
                reward += 0.1
            
            # Special reward for 'locking' a target (reaching Classification_6 and successfully completing)
            target_id = task_to_process['Target_ID']
            task_type = task_to_process['Task_Type']
            stage_count = task_to_process['Stage_Count']

            if task_type == 'Classification' and stage_count == self.MAX_STAGES_PER_TARGET_TYPE['Classification']:
                # This task completes the 'Classification' sequence for this target
                reward += 10.0 # Significant bonus for locking a target
                self.target_states[target_id]['is_locked'] = True
                self.target_states[target_id]['current_stage'] = 'Locked' # Update state

        else:
            reward -= 5.0 # Larger penalty for dropping a task
            self.dropped_tasks_in_episode += 1

        # Log the processed task
        self.scheduled_tasks_log.append({
            'Current_Sim_Time': self.current_simulation_time, # When the action was taken
            'Target_ID': task_to_process['Target_ID'],
            'Task_ID': task_to_process['Task_ID'],
            'Task_Type': task_to_process['Task_Type'],
            'Priority': task_to_process['Priority'],
            'Request_Time': task_to_process['Request_Time'],
            'Original_Deadline': task_to_process['Deadline'],
            'Original_Duration': task_to_process['Duration'],
            'Original_Init_Power': task_to_process['Init_Power'],
            'Original_Max_Power': task_to_process['Max_Power'],
            'Stage_Count': task_to_process['Stage_Count'],
            'Action_Delay': delay_action,
            'Action_Compress': compress_action,
            'Action_Radar': radar_choice_action,
            'Calculated_TNS': calculated_tns,
            'Calculated_Pn': calculated_Pn,
            'Task_Dropped': task_dropped,
            'Reward_Earned': reward
        })

        # Schedule a 'TASK_COMPLETION' event for this task, which will happen at `calculated_tns + Duration`
        # This event will then trigger the next task generation for the target if applicable
        # We need to make sure the completion time is based on the *actual* duration, not adjusted by action.
        # The agent chose a *start time*, not a completion time directly.
        completion_event_time = calculated_tns + task_to_process['Duration']
        
        if not task_dropped and completion_event_time < self.simulation_duration_minutes:
            heapq.heappush(self.event_queue, (completion_event_time, 'TASK_COMPLETION', {
                'target_id': task_to_process['Target_ID'],
                'task_type': task_to_process['Task_Type'],
                'stage_count': task_to_process['Stage_Count'] # Pass current stage count to completion handler
            }))

        # Advance simulation time after processing an action.
        # This also handles triggering new tasks from events that occur up to this point.
        self._advance_simulation_to_next_event()

        # Check for done condition:
        # 1. Simulation duration reached.
        # 2. All active targets are 'Locked'. (This is a strong condition, might prefer just time limit)
        # 3. No more tasks in task_queue and no more events in event_queue.
        
        if self.current_simulation_time >= self.simulation_duration_minutes:
            done = True
        elif not self.task_queue and not self.event_queue:
            # Only done if no more tasks are coming AND simulation time has passed
            done = True

        info = self._get_info()
        self._last_reward = reward # Store for logging

        obs = self._get_observation()
        return obs, reward, done, truncated, info

    def _get_observation(self):
        """
        Returns an observation based on the highest priority task currently in the queue.
        If no tasks, returns a zero observation.
        """
        if self.task_queue:
            # Peek at the top task (highest priority)
            _priority, _entry_count, task_dict = self.task_queue[0]
            self.current_task_for_observation = task_dict

            # Observation: [Duration, Deadline_Relative_to_Current_Time, Init_Power]
            # Deadline_Relative_to_Current_Time can be negative if it's already overdue.
            deadline_relative = task_dict['Deadline'] - self.current_simulation_time
            
            return np.array([task_dict['Duration'], deadline_relative, task_dict['Init_Power']], dtype=np.float32)
        else:
            self.current_task_for_observation = None
            # Return a default observation if no tasks are available
            return np.array([0, -self.simulation_duration_minutes, 0], dtype=np.float32)

    def _get_info(self):
        """Provides additional debugging/logging information."""
        info = {
            "current_simulation_time": self.current_simulation_time,
            "tasks_in_queue": len(self.task_queue),
            "events_in_queue": len(self.event_queue),
            "dropped_tasks_in_episode": self.dropped_tasks_in_episode,
            "completed_tasks_count": len(self.scheduled_tasks_log) - self.dropped_tasks_in_episode,
            "total_tasks_processed_this_step": len(self.scheduled_tasks_log),
            "reward_from_last_step": self._last_reward
        }
        if self.current_task_for_observation:
            info["observing_task_id"] = self.current_task_for_observation['Task_ID']
            info["observing_task_type"] = self.current_task_for_observation['Task_Type']
            info["observing_target_id"] = self.current_task_for_observation['Target_ID']
            info["observing_request_time"] = self.current_task_for_observation['Request_Time']
        else:
            info["observing_task_id"] = "N/A"
        return info


    def close(self):
        pass

# --- Episode Logger Callback (Adjusted for new info structure) ---
class EpisodeLoggerCallback(BaseCallback):
    def __init__(self, output_file, total_episodes_target, verbose=0):
        super(EpisodeLoggerCallback, self).__init__(verbose)
        self.output_file = output_file
        self.total_episodes_target = total_episodes_target
        self.episode_num = 0
        self.reward_log = []
        self.task_drop_log = []
        self.completed_tasks_log = []

    def _on_step(self) -> bool:
        if self.locals['dones'][0]:
            self.episode_num += 1
            
            last_step_info = self.locals['infos'][0]
            
            # Monitor wrapper adds 'episode' key to info dict
            if 'episode' in last_step_info:
                episode_stats = last_step_info['episode']
                total_reward_episode = episode_stats['r']
                
                # These metrics are now directly from the environment's info at the end of the episode
                task_drops_episode = last_step_info.get('dropped_tasks_in_episode', 0)
                completed_tasks_episode = last_step_info.get('completed_tasks_count', 0)
                
                self.reward_log.append(total_reward_episode)
                self.task_drop_log.append(task_drops_episode)
                self.completed_tasks_log.append(completed_tasks_episode)

                epsilon = self.model.exploration_rate if hasattr(self.model, 'exploration_rate') else 'N/A'

                log_message = (
                    f"Episode {self.episode_num}/{self.total_episodes_target} "
                    f"— Total Reward: {total_reward_episode:.2f} "
                    f"— Tasks Dropped: {task_drops_episode} "
                    f"— Tasks Completed: {completed_tasks_episode} "
                    f"— Sim Time: {last_step_info.get('current_simulation_time', 'N/A'):.2f} min "
                    f"— Epsilon (approx): {epsilon:.3f}"
                )
                print(log_message)
                self.output_file.write(log_message + "\n")
        return True

    def _on_rollout_end(self) -> None:
        pass

    def _on_training_end(self) -> None:
        pass

# --- Main Training and Evaluation Script ---
if __name__ == "__main__":
    # The 'task_file' parameter is no longer needed for the environment,
    # as it generates tasks dynamically.
    # The simulation_duration_minutes determines the length of an episode.
    SIM_DURATION = 120 # minutes per episode (e.g., 2 hours)

    # Create the base environment first
    base_env = RadarTaskSchedulerEnv(simulation_duration_minutes=SIM_DURATION, render_mode=None)
    # Wrap it with Monitor
    env = Monitor(base_env) # Monitor wraps the environment to log episode statistics

    dqn_hyperparams = {
        "learning_rate": 0.0005,
        "buffer_size": 50000, # Increased buffer size for continuous task flow
        "learning_starts": 2000, # Increased learning starts
        "batch_size": 64,
        "gamma": 0.99,
        "train_freq": (4, "step"),
        "gradient_steps": 1,
        "target_update_interval": 200,
        "exploration_fraction": 0.3, # Slightly longer exploration
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "max_grad_norm": 10,
        "seed": 42
    }

    model = DQN("MlpPolicy", env, verbose=0, **dqn_hyperparams, device="auto")

    # For dynamic environments, total_timesteps is usually the main stopping criterion.
    # Each 'step' processes one task or advances time.
    # A single episode runs for SIM_DURATION.
    # Let's estimate how many steps per episode:
    # Avg task duration ~ 4 min. Avg inter-arrival ~ 5 min. So, tasks arrive somewhat continuously.
    # Number of steps = total simulation time / average time per task process.
    # This is hard to estimate, so we'll just set a large total_timesteps.
    # A better approach might be to train for a fixed number of episodes, and let each episode
    # run until its SIM_DURATION is met.
    
    episodes_to_train = 1000 # Train for 1000 full simulation durations
    # total_timesteps will be the *maximum* possible steps.
    # Assuming roughly 1 task scheduled per minute on average if active.
    # So, SIM_DURATION * (approx tasks per minute) * episodes_to_train
    # Let's say 0.5 tasks per minute (due to processing time + gaps) = SIM_DURATION * 0.5
    total_timesteps_for_training = episodes_to_train * SIM_DURATION * 1 # Assuming roughly 1 step per simulation minute on average

    output_filename = "dqn_learning_results_dynamic_event_driven.txt"
    
    with open(output_filename, "w") as f_out:
        f_out.write("--- DQN Training Log (Dynamic Event-Driven Tasks) ---\n\n")
        print("Starting DQN training with dynamic event-driven task generation...")
        f_out.write("Starting DQN training with dynamic event-driven task generation...\n\n")

        episode_logger_callback = EpisodeLoggerCallback(
            output_file=f_out,
            total_episodes_target=episodes_to_train
        )

        model.learn(total_timesteps=total_timesteps_for_training, callback=episode_logger_callback)

        print("\nTraining complete!")
        f_out.write("\nTraining complete!\n")

        model_path = "dqn_radar_scheduler_dynamic_event_driven_model"
        model.save(model_path)
        print(f"Final model saved to {model_path}")
        f_out.write(f"Final model saved to {model_path}\n")

        # --- Plotting Results ---
        reward_log = episode_logger_callback.reward_log
        task_drop_log = episode_logger_callback.task_drop_log
        completed_tasks_log = episode_logger_callback.completed_tasks_log

        window_size = 50 # For moving average

        # Plot Rewards
        if reward_log:
            plt.figure(figsize=(14,7))
            plt.plot(reward_log, label='Episode Rewards', color='orange', alpha=0.7, linewidth=1)
            if len(reward_log) >= window_size:
                moving_avg = np.convolve(reward_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg = range(window_size - 1, len(reward_log))
                plt.plot(x_moving_avg, moving_avg, label=f'Moving Average ({window_size} episodes)', color='blue', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Total Reward', fontsize=12)
            plt.title('Total Reward over Episodes (DQN - Dynamic Event-Driven)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_reward_plot_dynamic_event_driven.png")
            plt.close()
            print(f"Reward plot saved as dqn_reward_plot_dynamic_event_driven.png")
            f_out.write(f"Reward plot saved as dqn_reward_plot_dynamic_event_driven.png\n")
        else:
            print("\nReward log is empty. No reward plot will be generated.")
            f_out.write("\nReward log is empty. No reward plot will be generated.\n")

        # Plot Tasks Dropped
        if task_drop_log:
            plt.figure(figsize=(14,7))
            plt.plot(task_drop_log, label='Tasks Dropped per Episode', color='red', alpha=0.8, linewidth=1)
            if len(task_drop_log) >= window_size:
                moving_avg_drops = np.convolve(task_drop_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_drops = range(window_size - 1, len(task_drop_log))
                plt.plot(x_moving_avg_drops, moving_avg_drops, label=f'Moving Average ({window_size} episodes)', color='darkred', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Dropped', fontsize=12)
            plt.title('Number of Tasks Dropped per Episode (DQN - Dynamic Event-Driven)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_task_drop_plot_dynamic_event_driven.png")
            plt.close()
            print(f"Task drop plot saved as dqn_task_drop_plot_dynamic_event_driven.png")
            f_out.write(f"Task drop plot saved as dqn_task_drop_plot_dynamic_event_driven.png\n")
        else:
            print("\nTask drop log is empty. No task drop plot will be generated.")
            f_out.write("\nTask drop log is empty. No task drop plot will be generated.\n")

        # Plot Tasks Completed
        if completed_tasks_log:
            plt.figure(figsize=(14,7))
            plt.plot(completed_tasks_log, label='Tasks Completed per Episode', color='green', alpha=0.8, linewidth=1)
            if len(completed_tasks_log) >= window_size:
                moving_avg_completed = np.convolve(completed_tasks_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_completed = range(window_size - 1, len(completed_tasks_log))
                plt.plot(x_moving_avg_completed, moving_avg_completed, label=f'Moving Average ({window_size} episodes)', color='darkgreen', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Completed', fontsize=12)
            plt.title('Number of Tasks Completed per Episode (DQN - Dynamic Event-Driven)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_task_completed_plot_dynamic_event_driven.png")
            plt.close()
            print(f"Task completed plot saved as dqn_task_completed_plot_dynamic_event_driven.png")
            f_out.write(f"Task completed plot saved as dqn_task_completed_plot_dynamic_event_driven.png\n")
        else:
            print("\nTask completed log is empty. No task completed plot will be generated.")
            f_out.write("\nTask completed log is empty. No task completed plot will be generated.\n")


        f_out.write("\n\n--- Running Final Evaluation Episode for Schedule and Metrics ---\n")
        print("\n--- Running Final Evaluation Episode for Schedule and Metrics ---")

        # Use a fresh environment for evaluation
        eval_env = RadarTaskSchedulerEnv(simulation_duration_minutes=SIM_DURATION, render_mode=None)
        
        obs, _ = eval_env.reset()
        done = False
        
        while not done:
            action_index, _states = model.predict(obs, deterministic=True) # Use deterministic actions for evaluation
            obs, reward, done, _, info = eval_env.step(action_index)

        f_out.write("\n--- Final Task Schedule (from last evaluation) ---\n")
        print("\n--- Final Task Schedule (from last evaluation) ---")
        if eval_env.scheduled_tasks_log:
            schedule_df = pd.DataFrame(eval_env.scheduled_tasks_log)
            # Sort by Current_Sim_Time to see the chronological order of scheduling
            schedule_df_sorted = schedule_df.sort_values(by='Current_Sim_Time').reset_index(drop=True)
            f_out.write(schedule_df_sorted.to_string(index=False))
            print(schedule_df_sorted.to_string(index=False))
        else:
            f_out.write("No tasks processed in the final evaluation. This might mean the environment did not start a full episode.\n")
            print("No tasks processed in the final evaluation. This might mean the environment did not start a full episode.")


        f_out.write("\n\n--- Performance Metrics (from last evaluation) ---\n")
        print("\n--- Performance Metrics (from last evaluation) ---")

        if eval_env.scheduled_tasks_log:
            total_tasks_attempted = len(eval_env.scheduled_tasks_log)
            completed_tasks = sum(1 for task in eval_env.scheduled_tasks_log if not task['Task_Dropped'])
            dropped_tasks_count = sum(1 for task in eval_env.scheduled_tasks_log if task['Task_Dropped'])
            
            # Count locked targets based on eval_env's final state
            locked_targets_count = sum(1 for state in eval_env.target_states.values() if state['is_locked'])
            
            # This is total unique targets that *appeared* in this specific eval episode
            unique_targets_in_eval = len(eval_env.target_states)


            if total_tasks_attempted > 0:
                percent_tasks_completed = (completed_tasks / total_tasks_attempted) * 100
                percent_tasks_dropped = (dropped_tasks_count / total_tasks_attempted) * 100
            else:
                percent_tasks_completed = 0.0
                percent_tasks_dropped = 0.0

            metrics = {
                "Simulation Duration (min)": SIM_DURATION,
                "Total Unique Targets Appeared (in this eval episode)": unique_targets_in_eval,
                "Targets Successfully Locked (Completed all stages)": locked_targets_count,
                "Total Tasks Attempted (in last evaluation)": total_tasks_attempted,
                "Tasks Successfully Completed (Individual)": completed_tasks,
                "Tasks Dropped (Individual)": dropped_tasks_count,
                "Percentage of Tasks Completed (Individual)": f"{percent_tasks_completed:.2f}%",
                "Percentage of Tasks Dropped (Individual)": f"{percent_tasks_dropped:.2f}%"
            }

            metrics_df = pd.DataFrame(metrics.items(), columns=['Metric', 'Value'])
            f_out.write(metrics_df.to_string(index=False))
            print(metrics_df.to_string(index=False))
        else:
            f_out.write("No metrics could be calculated from the final evaluation as no tasks were processed.\n")
            print("No metrics could be calculated from the final evaluation as no tasks were processed.")

    env.close()
    eval_env.close()
    print("All environments closed.")

Starting DQN training with dynamic event-driven task generation...
Episode 1/1000 — Total Reward: -13.38 — Tasks Dropped: 21 — Tasks Completed: 56 — Sim Time: 120.00 min — Epsilon (approx): 0.998
Episode 2/1000 — Total Reward: -39.20 — Tasks Dropped: 23 — Tasks Completed: 46 — Sim Time: 120.00 min — Epsilon (approx): 0.996
Episode 3/1000 — Total Reward: -17.88 — Tasks Dropped: 20 — Tasks Completed: 49 — Sim Time: 120.00 min — Epsilon (approx): 0.994
Episode 4/1000 — Total Reward: 11.39 — Tasks Dropped: 24 — Tasks Completed: 79 — Sim Time: 120.00 min — Epsilon (approx): 0.992
Episode 5/1000 — Total Reward: -37.43 — Tasks Dropped: 19 — Tasks Completed: 34 — Sim Time: 120.00 min — Epsilon (approx): 0.990
Episode 6/1000 — Total Reward: -27.58 — Tasks Dropped: 24 — Tasks Completed: 55 — Sim Time: 120.00 min — Epsilon (approx): 0.988
Episode 7/1000 — Total Reward: -52.64 — Tasks Dropped: 26 — Tasks Completed: 46 — Sim Time: 120.00 min — Epsilon (approx): 0.986
Episode 8/1000 — Total Reward: 